In [1]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Build DB URL
DATABASE_URL = os.getenv("DATABASE_URL") or (
    f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}"
    f"@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)

def test_connection():
    try:
        engine = create_engine(DATABASE_URL)
        with engine.connect() as conn:
            result = conn.execute(text("SELECT 1")).scalar()
            if result == 1:
                print("✔ Connection Successful! DB is reachable.")
    except Exception as e:
        print(f"❌ Connection Failed:\n{e}")

if __name__ == "__main__":
    test_connection()


✔ Connection Successful! DB is reachable.


# Data Check

In [6]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL") or (
    f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@"
    f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)

engine = create_engine(DATABASE_URL)

df = pd.read_sql("SELECT * FROM finance_data order by period DESC", engine)

# Show everything
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(10)


,period,accounts,category,amount,month,_row,running_total
0,2025-12-27 00:00:00,UPI,Food,15.0,None,NaN,NaN
1,2025-12-27 00:00:00,UPI,Food,50.0,None,NaN,NaN
2,2025-12-27 00:00:00,UPI,Other,85.0,None,NaN,NaN
3,2025-12-25 00:00:00,Upi,Transport,227.0,2025-12,0.065816,344899.96
4,2025-12-24 00:00:00,Upi,Party,144.0,2025-12,0.041846,344120.96
5,2025-12-24 00:00:00,Upi,Food,182.0,2025-12,0.052860,344302.96
6,2025-12-24 00:00:00,Upi,Gift,250.0,2025-12,0.072558,344552.96
7,2025-12-24 00:00:00,Upi,Food,34.0,2025-12,0.009884,343976.96
8,2025-12-23 00:00:00,Upi,Food,101.0,2025-12,0.029365,343942.96
9,2025-12-23 00:00:00,Upi,Gift,120.0,2025-12,0.034816,344672.96


In [23]:
engine = create_engine(DATABASE_URL)

df = pd.read_sql("SELECT period,category, AVG(amount) as avg_amount FROM finance_data GROUP BY category,period Having category = 'Rent' ORDER BY avg_amount DESC", engine)

# Show everything
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(25)


,period,category,avg_amount
0,2024-10-02 00:00:00,Rent,46000.0
1,2025-04-01 00:00:00,Rent,11686.0
2,2025-07-02 00:00:00,Rent,11679.0
3,2025-05-02 00:00:00,Rent,11670.0
4,2025-06-01 00:00:00,Rent,11649.0
5,2025-03-03 00:00:00,Rent,11596.0
6,2025-01-01 00:00:00,Rent,11517.0
7,2025-02-01 00:00:00,Rent,11498.0
8,2024-11-02 00:00:00,Rent,11400.0
9,2024-12-01 00:00:00,Rent,11400.0


In [11]:
engine = create_engine(DATABASE_URL)

df = pd.read_sql("SELECT period,category, AVG(amount) as avg_amount FROM finance_data GROUP BY period, category ORDER BY avg_amount DESC", engine)

# Show everything
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(25)


,period,category,avg_amount
0,2024-10-02 00:00:00,Rent,46000.0
1,2025-04-01 00:00:00,Rent,11686.0
2,2025-07-02 00:00:00,Rent,11679.0
3,2025-05-02 00:00:00,Rent,11670.0
4,2025-06-01 00:00:00,Rent,11649.0
5,2025-03-03 00:00:00,Rent,11596.0
6,2025-01-01 00:00:00,Rent,11517.0
7,2025-02-01 00:00:00,Rent,11498.0
8,2024-11-02 00:00:00,Rent,11400.0
9,2024-12-01 00:00:00,Rent,11400.0


In [17]:
print(3*(11686- 9800))

print(3*(11686- 6500))

print(((11686- 6500)/(11686- 9800)))


5658
15558
2.7497348886532342


# Recovery

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# ============================================================
# 🔐 Load Environment Variables
# ============================================================
load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL") or (
    f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@"
    f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)

# ============================================================
# 📂 Excel File Path
# ============================================================
file_path = r"C:\Users\trilo\OneDrive\Desktop\Report_Transactions\3380.xlsx"

# ============================================================
# 🗄 Table Name (change if needed)
# ============================================================
TABLE_NAME = "finance_data"

# ============================================================
# 🚀 Push Excel → PostgreSQL
# ============================================================
def push_excel_to_db():
    # Read Excel
    df = pd.read_excel(file_path)

    # Optional: clean column names (recommended)
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace(r"[^\w_]", "", regex=True)
    )

    # Create DB engine
    engine = create_engine(DATABASE_URL)

    # Push to DB
    df.to_sql(
        TABLE_NAME,
        engine,
        if_exists="append",   # use "replace" if you want to overwrite
        index=False,
        method="multi",
        chunksize=5000
    )

    print(f"✅ {len(df)} rows pushed successfully to '{TABLE_NAME}'")

# ============================================================
# ▶ Run
# ============================================================
if __name__ == "__main__":
    push_excel_to_db()


In [ ]:
import subprocess

PG_BIN = r"C:\Program Files\PostgreSQL\16\bin"
CREATEDB = fr"{PG_BIN}\createdb.exe"
PG_RESTORE = fr"{PG_BIN}\pg_restore.exe"

BACKUP_FILE = r"C:\Users\trilo\OneDrive\Desktop\Report_Transactions\Backup\3380.dat"
DB_NAME = "report_mqba_restore"

subprocess.run([CREATEDB, DB_NAME], capture_output=True)
subprocess.run([PG_RESTORE, "-d", DB_NAME, BACKUP_FILE], check=True)

print("✅ Database restored")
